In [1]:
import numpy as np
import sympy as sy
import scipy as sc

import matplotlib.pyplot as plt

import cirq


##### **!! NOTE: This python + cirq notebook is legacy code for reference. It should work fine but the Julia + Yao Edition is much more up-to-date and tested. All relevant experiments have been performed using the Julia Edition of this notebook, *not* this one. !!**

In [2]:
def prepare_loss_gradient_approx(loss_func, stochastic_approx=False, stoch_shift_size_c=0.05):
    
    def loss_grad_func(eval_point, *loss_func_args):
        """
        Returns the gradient of the loss function at the given point.
        If stochastic_approx is True, then the gradient is estimated using the SPSA rule.
        """
        
        loss_grad = []
        #eval_point = np.array(eval_point)
        
        if not stochastic_approx: #Non-stochastic parameter shift rule
            for input_dim in range(len(eval_point)):
                dim_basis_vec = np.zeros(len(eval_point))
                dim_basis_vec[input_dim] = 1.0
                loss_grad.append((loss_func(eval_point + np.pi*dim_basis_vec/2, *loss_func_args) - loss_func(eval_point - np.pi*dim_basis_vec/2, *loss_func_args))/2)
            loss_grad = np.array(loss_grad)
        else: #SPSA
            random_shift_vec = np.random.choice([-1,1],len(eval_point))
            loss_grad = (loss_func(eval_point + stoch_shift_size_c*random_shift_vec, *loss_func_args) - loss_func(eval_point - stoch_shift_size_c*random_shift_vec, *loss_func_args))/(2*stoch_shift_size_c*random_shift_vec)
        
        return loss_grad
    return loss_grad_func

### The Meta-VQE

In [3]:
def processing_step_circuit(qubits, thetas):
    """
    Returns a circuit that applies the processing step of the VQE algorithm.
    
    Args:
    ----------
        qubits [list]: Qubits to apply the circuit to.
        thetas [list or ndarray]: A list of the variational parameters.
    
    Yields:
    ----------
        [list]: Gates in the circuit.
    """
    assert len(thetas) == 2 * 2 * len(qubits) # 2 paramterized layers per qubit with 2 rotation gates each
    
    rz_thetas = thetas[0:2*len(qubits)]
    ry_thetas = thetas[2*len(qubits):4*len(qubits)]
    
    #Variational Layer 1:
    for q_i, _ in enumerate(qubits):
        yield [cirq.rz(rz_thetas[2*q_i])(qubits[q_i]), cirq.ry(ry_thetas[2*q_i])(qubits[q_i])] # Even indices in the theta vectors are for the first variational layer
    
    # In-between Entangling Layer (even control indices):
    yield [cirq.CNOT(qubits[q_i], qubits[q_i+1]) for q_i in range(0, len(qubits)-1, 2)]
    
    #Variational Layer 2:
    for q_i, _ in enumerate(qubits):
        yield [cirq.rz(rz_thetas[2*q_i + 1])(qubits[q_i]), cirq.ry(ry_thetas[2*q_i + 1])(qubits[q_i])] # Odd indices in the theta vectors are for the second variational layer
    
    # Final Entangling Layer (odd control indices):
    yield [cirq.CNOT(qubits[q_i], qubits[q_i+1]) for q_i in range(1, len(qubits)-1, 2)]


In [4]:
import sympy

num_qubits = 4
qubits = cirq.LineQubit.range(num_qubits)

theta_0 = [sympy.Symbol("theta_{}^({})".format(i, j)) for j in range(2) for i in range(2 * num_qubits)]

circuit = cirq.Circuit(processing_step_circuit(qubits, theta_0))
print(circuit.to_text_diagram(transpose=True))

0               1               2               3
│               │               │               │
Rz(theta_0^(0)) Rz(theta_2^(0)) Rz(theta_4^(0)) Rz(theta_6^(0))
│               │               │               │
Ry(theta_0^(1)) Ry(theta_2^(1)) Ry(theta_4^(1)) Ry(theta_6^(1))
│               │               │               │
@───────────────X               @───────────────X
│               │               │               │
Rz(theta_1^(0)) Rz(theta_3^(0)) Rz(theta_5^(0)) Rz(theta_7^(0))
│               │               │               │
Ry(theta_1^(1)) Ry(theta_3^(1)) Ry(theta_5^(1)) Ry(theta_7^(1))
│               │               │               │
│               @───────────────X               │
│               │               │               │


In [5]:
def encoding_function(Delta, phi):
    """
    Returns the value of the encoding function for a given Delta and input vector phi
    
    Args:
    ----------
        Delta [float]: The value of the parameter Delta.
        phi [list or ndarray]: A list of the variational parameters.
    
    Returns:
    ----------
        [ndarray]: The encoding function.
    """
    weights = phi[:int(len(phi)/2)]
    biases = phi[int(len(phi)/2):]
    weights_z, weights_y = weights[:int(len(weights)/2)], weights[int(len(weights)/2):]
    biases_z, biases_y = biases[:int(len(biases)/2)], biases[int(len(biases)/2):]
    
    return np.concatenate((weights_z * Delta + biases_z, weights_y * Delta + biases_y))

def encoding_step_circuit(qubits, phi, Delta, encoding_function):
    """
    Returns a circuit that applies the encoding step of the VQE algorithm.
    
    Args:
    ----------
        qubits [list]: Qubits to apply the circuit to.
        phi [list or ndarray]: A list of the variational parameters.
        Delta [float]: The value of the parameter Delta.
        encoding_function [function]: The encoding function.
    
    Yields:
    ----------
        [list]: Gates in the circuit.
    """
    
    assert len(phi) == 2 * 2 * 2 * len(qubits) # 2 paramterized layers per qubit with 2 fun
    
    yield processing_step_circuit(qubits, encoding_function(Delta, phi))

In [6]:
import sympy

num_qubits = 4
qubits = cirq.LineQubit.range(num_qubits)

phi_0 = np.array([sympy.Symbol("{}_{}^({})".format(phi_type, i, j)) for phi_type in ["w", "p"] for j in range(2) for i in range(2 * num_qubits)])

circuit = cirq.Circuit(encoding_step_circuit(qubits, phi_0, Delta=sympy.Symbol("Delta") ,encoding_function=encoding_function))
print(circuit.to_text_diagram(transpose=True))

0                           1                           2                           3
│                           │                           │                           │
Rz(Delta*w_0^(0) + p_0^(0)) Rz(Delta*w_2^(0) + p_2^(0)) Rz(Delta*w_4^(0) + p_4^(0)) Rz(Delta*w_6^(0) + p_6^(0))
│                           │                           │                           │
Ry(Delta*w_0^(1) + p_0^(1)) Ry(Delta*w_2^(1) + p_2^(1)) Ry(Delta*w_4^(1) + p_4^(1)) Ry(Delta*w_6^(1) + p_6^(1))
│                           │                           │                           │
@───────────────────────────X                           @───────────────────────────X
│                           │                           │                           │
Rz(Delta*w_1^(0) + p_1^(0)) Rz(Delta*w_3^(0) + p_3^(0)) Rz(Delta*w_5^(0) + p_5^(0)) Rz(Delta*w_7^(0) + p_7^(0))
│                           │                           │                           │
Ry(Delta*w_1^(1) + p_1^(1)) Ry(Delta*w_3^(1) + p_3^(1)) Ry(Del

In [7]:

def xxz_chain_hamiltonian_1D(qubits, hamilt_params):
    """
    Returns tuple of generators for every hamiltonian term in the XXZ chain hamiltonian
    such that each inidivdual term is given as (coefficient, gate set)
    
    Args:
    ----------
        qubits [list]: Qubits to apply the circuit to.
        hamilt_params [list]: A list of the hamiltonian parameters.
        
    Returns:
    ----------
        [tuple]: Tuple of tuples of hamiltonian term coefficient and gate generators.
    """
    
    def x_interact_term():
        for q_i, _ in enumerate(qubits):
            yield cirq.X(qubits[q_i]) * cirq.X(qubits[(q_i+1) % len(qubits)])
    
    def y_interact_term():
        for q_i, _ in enumerate(qubits):
            yield cirq.Y(qubits[q_i]) * cirq.Y(qubits[(q_i+1) % len(qubits)])
    
    def z_interact_term():
        for q_i, _ in enumerate(qubits):
            yield cirq.Z(qubits[q_i]) * cirq.Z(qubits[(q_i+1) % len(qubits)])
    
    def z_align_term():
        for q_i, _ in enumerate(qubits):
            yield cirq.Z(qubits[q_i])

    return (1.0, x_interact_term()), (1.0, y_interact_term()),  (hamilt_params["Delta"],z_interact_term()), (hamilt_params["lambda"], z_align_term())

def get_XXZ_hamiltonian_measurement_basis_change(qubits, hamilt_params):
    """
    Returns the gates for the measurement basis change needed to measure each commuting set of terms in the XXZ chain hamiltonian.
    Equivalent to applying to every pauli-string the .to_z_basis_op() method.
    Note that the following code is generalizable to other hamiltonians
    However, the iteration over terms (i.e. term_tuples) assumes that all pauli strings within each term commute with each other
    
    Args:
    ----------
        qubits [list]: Qubits to apply the circuit to.
        hamilt_params [list]: A list of the hamiltonian parameters.
    
    Yields:
    ----------
        [list]: Gates in the circuit.
    """	
    
    term_tuples = xxz_chain_hamiltonian_1D(qubits, hamilt_params)
    
    for term_coeff, term_generator in term_tuples:
        
        measured_strings = [] # list of pauli operator objects
        term_measurement_gateset = []
        measured_qubits = []
        
        for pauli_str in term_generator:
            measured_strings.append(pauli_str)
            for qubit, operator in pauli_str.items():
                if operator == cirq.X and qubit not in measured_qubits:
                    term_measurement_gateset.append(cirq.H(qubit))
                    measured_qubits.append(qubit)
                elif operator == cirq.Y and qubit not in measured_qubits:
                    term_measurement_gateset.append(cirq.inverse(cirq.S(qubit)))
                    term_measurement_gateset.append(cirq.H(qubit))
                    measured_qubits.append(qubit)

        yield (term_coeff, measured_strings), term_measurement_gateset

#def get_hamilt_term_fold_func(term_pauli_strings):
#    def term_fold_func(meas_results):
#        #Change from binary to -1 and 1
#        meas_results = 1 - 2 * np.array([meas_results[q_i] for q_i in range(len(meas_results))]).astype(int)
#        
#        energy_tot = np.sum([np.prod([meas_results[q_i] for q_i, _ in enumerate(pauli_str.qubits)]) for pauli_str in term_pauli_strings])
#        
#        return energy_tot
#    return term_fold_func

def meas_hamilt_expectation(qubits, circuit_params, hamilt_params, n_meas_reps = 1000):
    """
    Returns the expectation value of the hamiltonian on the state defined by the given circuit parameters.
    Non-cheating version, i.e. the state vector is not computed but instead the expectation value is estimated by sampling measurement outcomes
    
    Args:
    ----------
        qubits [list]: Qubits to apply the circuit to.
        circuit_params [list]: A list of the circuit parameters.
        hamilt_params [list]: A list of the hamiltonian parameters.
        n_meas_reps [int]: Number of measurement repetitions.
    
    Returns:
    ----------
        [float]: Expectation value of the hamiltonian.
    
    """
    energy_expect = 0.0
    
    #Iterate over terms with commuting pauli strings in the hamiltonian
    for measured_term, term_basis_change_gates in get_XXZ_hamiltonian_measurement_basis_change(qubits, hamilt_params):
        
        term_coeff, measured_strings = measured_term

        #Note that we have to re-initialize the circuit for each term, since those are generators
        encoding_step = encoding_step_circuit(qubits, circuit_params[:int(8*len(qubits))], Delta=hamilt_params['Delta'] ,encoding_function=encoding_function)
        processing_step = processing_step_circuit(qubits, circuit_params[int(8*len(qubits)):])

        #Prepare, run and measure the circuit 
        term_basis_change_gates.extend([cirq.measure(q, key=f"%d" % q.x) for q in qubits])
        
        #OR: term_basis_change_gates = cirq.measure_paulistring_terms(cirq.PauliString({q_i: measured_strings[0]._qubit_pauli_map[qubits[0]] for q_i in qubits}), key_func = lambda q: f"%d" % q.x)

        #print("%%%%%%%%%%%%%%%%%% Next %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        term_meas_circuit = cirq.Circuit([encoding_step,
                                    processing_step,
                                    term_basis_change_gates,
                                    ])
        
        #print(term_meas_circuit.to_text_diagram(transpose=True))
        
        #print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        
        
        simulator = cirq.Simulator()
        results = simulator.run(term_meas_circuit, repetitions=int(n_meas_reps))
        
        meas_outcomes_hist = results.multi_measurement_histogram(keys=[f"%d" % q.x for q in qubits])
        assert n_meas_reps == int(np.sum(list(meas_outcomes_hist.values())))
        
        #print("Finished term measurements: ", meas_outcomes_hist)
        
        term_expect = 0.0
        for pauli_str in measured_strings:
            pauli_str_expect = 0.0
            for meas_outcomes, count in meas_outcomes_hist.items():
                assert len(meas_outcomes) == len(qubits)
                pauli_str_eigval = np.prod([1 - 2 * meas_outcomes[q_i.x] for q_i in pauli_str.qubits])
                #print(pauli_str.qubits,[q_i.x for q_i in pauli_str.qubits],[meas_outcomes[q_i.x] for q_i in pauli_str.qubits], [1 - 2 * meas_outcomes[q_i.x] for q_i in pauli_str.qubits])
                pauli_str_expect += pauli_str_eigval * count/n_meas_reps
            
            term_expect += pauli_str_expect
            
        energy_expect += term_coeff * term_expect
    
    return energy_expect

def direct_hamilt_expectation(qubits, circuit_params, hamilt_params):
    """
    Returns the expectation value of the hamiltonian on the state defined by the given circuit parameters.
    Cheating version, i.e. the state vector is computed exactly and the hamiltonian matrix representation is used to compute the expectation value directly; no sampling noise.
    
    Args:
    ----------
        qubits [list]: Qubits to apply the circuit to.
        circuit_params [list]: A list of the circuit parameters.
        hamilt_params [list]: A list of the hamiltonian parameters.
    
    Returns:
    ----------
        [float]: Expectation value of the hamiltonian.
    """
    
    energy_expect = 0.0

    #Iterate over terms with commuting pauli strings in the hamiltonian
    for measured_term, term_basis_change_gates in get_XXZ_hamiltonian_measurement_basis_change(qubits, hamilt_params):
        
        term_coeff, measured_strings = measured_term

        #Note that we have to re-initialize the circuit for each term, since those are generators
        encoding_step = encoding_step_circuit(qubits, circuit_params[:int(8*len(qubits))], Delta=hamilt_params['Delta'] ,encoding_function=encoding_function)
        processing_step = processing_step_circuit(qubits, circuit_params[int(8*len(qubits)):])

        pqc_circuit = cirq.Circuit([encoding_step, processing_step])
        expval = lambda state: np.real(sum([state.conj() @ pauli_str.matrix(qubits) @ state for pauli_str in measured_strings]))
        
        simulator = cirq.Simulator()
        fin_state_vec = simulator.simulate(pqc_circuit, qubit_order=[*qubits]).final_state_vector
        #print("Term energy expect: ",expval(fin_state_vec), " for state vector: ", fin_state_vec)
        energy_expect += term_coeff*expval(fin_state_vec)
    
    return energy_expect


def exact_hamilt_GS_energy(qubits, hamilt_params):
    """
    Returns the exact ground state energy of the hamiltonian.
    
    Args:
    ----------
        qubits [list]: Qubits to apply the circuit to.
        hamilt_params [list]: A list of the hamiltonian parameters.
    
    Returns:
    ----------
        [float]: Exact ground state energy of the hamiltonian.
    """
    
    hamilt_matrix = np.zeros((2**len(qubits), 2**len(qubits)), dtype=np.complex128)
    
    term_tuples = xxz_chain_hamiltonian_1D(qubits, hamilt_params)
    for term_coeff, term_generator in term_tuples:
        for pauli_str in term_generator:
            hamilt_matrix += term_coeff*pauli_str.matrix(qubits)
    
    eigvals = np.linalg.eigvalsh(hamilt_matrix)
    gs_energy = np.min(eigvals)
    
    return gs_energy




In [8]:

import sympy

num_qubits = 3
qubits = cirq.LineQubit.range(num_qubits)

hamilt_params = {}
hamilt_params["lambda"] =  2.0  # Transverse field strength
hamilt_params["Delta"] = 2.0 # Anisotropy parameter

print("############ Direct Hamiltonian Circuits ############")

for term_coeff, term_generator in xxz_chain_hamiltonian_1D(qubits, hamilt_params):
    print("\nThe following term has coefficient", term_coeff, "\n")
    circuit = cirq.Circuit(term_generator)
    print(circuit.to_text_diagram(transpose=True))

print("\n############ Measurement Basis Change Circuits ############")

for measured_term, term_basis_change_gates in get_XXZ_hamiltonian_measurement_basis_change(qubits, hamilt_params):
    term_coeff, measured_strings = measured_term
    print("\nThe following term has coefficient", term_coeff, "\n")
    term_basis_change_gates.append(cirq.measure(*qubits, key='Zmeasurements'))
    circuit = cirq.Circuit(term_basis_change_gates)
    print(circuit.to_text_diagram(transpose=True))

############ Direct Hamiltonian Circuits ############

The following term has coefficient 1.0 

0               1               2
│               │               │
PauliString(+X)─X               │
│               │               │
│               PauliString(+X)─X
│               │               │
X───────────────┼───────────────PauliString(+X)
│               │               │

The following term has coefficient 1.0 

0               1               2
│               │               │
PauliString(+Y)─Y               │
│               │               │
│               PauliString(+Y)─Y
│               │               │
Y───────────────┼───────────────PauliString(+Y)
│               │               │

The following term has coefficient 2.0 

0               1               2
│               │               │
PauliString(+Z)─Z               │
│               │               │
│               PauliString(+Z)─Z
│               │               │
Z───────────────┼───────────────PauliString(

In [9]:
num_qubits = 8
qubits = cirq.LineQubit.range(num_qubits)

hamilt_params = {}
hamilt_params["lambda"] =  0.75  # Transverse field strength
hamilt_params["Delta"] = 0.5 # Anisotropy parameter

exact_hamilt_GS_energy(qubits, hamilt_params)

-12.487918414869853

In [10]:
#np.random.seed(0)
#np.random.seed(7) #43
init_phi_params = np.random.normal(0, np.pi/2, 8*len(qubits))	
init_theta_params = np.random.normal(0, np.pi/2, 4*len(qubits))
init_circuit_params = np.concatenate([init_phi_params, init_theta_params])

print(meas_hamilt_expectation(qubits, init_circuit_params, hamilt_params,100000))
print(direct_hamilt_expectation(qubits, init_circuit_params, hamilt_params))

0.19441500000000017
0.19132160504954204


### Construct training set of hamiltonian expectation values


In [57]:

def construct_training_set(train_set_size, min_hamilt_params = {"lambda": 0.75, "Delta": -1.1}, max_hamilt_params = {"lambda": 0.75, "Delta": 1.1}, mode="equidistant"):
    """
    Constructs a training set of hamiltonian parameters.
    
    Args:
    ----------
        train_set_size [int]: Size of the training set.
        min_hamilt_params [dict]: Dictionary of the minimum hamiltonian parameters.
        max_hamilt_params [dict]: Dictionary of the maximum hamiltonian parameters.
        mode [str]: Mode of the training set construction. Either "equidistant" or "uniform_random".
    
    Returns:
    ----------
        [list]: List of hamiltonian parameters.
    """
    
    assert min_hamilt_params.keys() == max_hamilt_params.keys()

    training_set = []
    for i in range(train_set_size):
        sample_i = {}
        for param in min_hamilt_params.keys():
            if mode == "equidistant":
                sample_i[param] = min_hamilt_params[param] + i*(max_hamilt_params[param] - min_hamilt_params[param])/(train_set_size-1)
            elif mode == "uniform_random": 
                sample_i[param] = np.random.uniform(min_hamilt_params[param], max_hamilt_params[param])
        
        training_set.append(sample_i)
    
    return training_set   
 
def construct_test_set(test_set_size, min_hamilt_params = {"lambda": 0.75, "Delta": -1.1}, max_hamilt_params = {"lambda": 0.75, "Delta": 1.1}, mode="uniform_random"):
    """
    Constructs a test set of hamiltonian parameters.
    
    Args:
    ----------
        test_set_size [int]: Size of the test set.
        min_hamilt_params [dict]: Dictionary of the minimum hamiltonian parameters.
        max_hamilt_params [dict]: Dictionary of the maximum hamiltonian parameters.
        mode [str]: Mode of the test set construction. Either "equidistant" or "uniform_random".
    
    Returns:
    ----------
        [list]: List of hamiltonian parameters.
    """
    
    return construct_training_set(test_set_size, min_hamilt_params, max_hamilt_params, mode)




In [58]:

train_set_size = 10
min_hamilt_params = {"lambda": 0.75, "Delta": -1.1}
max_hamilt_params = {"lambda": 0.75, "Delta": 1.1}

print(construct_training_set(train_set_size=train_set_size))

[{'lambda': 0.75, 'Delta': -1.1}, {'lambda': 0.75, 'Delta': -0.8555555555555556}, {'lambda': 0.75, 'Delta': -0.6111111111111112}, {'lambda': 0.75, 'Delta': -0.3666666666666667}, {'lambda': 0.75, 'Delta': -0.12222222222222223}, {'lambda': 0.75, 'Delta': 0.12222222222222223}, {'lambda': 0.75, 'Delta': 0.3666666666666667}, {'lambda': 0.75, 'Delta': 0.6111111111111112}, {'lambda': 0.75, 'Delta': 0.8555555555555556}, {'lambda': 0.75, 'Delta': 1.1}]


In [59]:
from scipy.optimize import OptimizeResult
def adam(
    fun,
    x0,
    jac,
    args=(),
    learning_rate=0.04, #0.001,
    beta1=0.9,
    beta2=0.999,
    eps=1e-8,
    startiter=0,
    maxiter=10000,
    callback=None,
    **kwargs):
    """scipy.optimize.minimize compatible implementation of ADAM - [http://arxiv.org/pdf/1412.6980.pdf].
    Adapted from autograd/misc/optimizers.py.
    """
    
    x = x0
    m = np.zeros_like(x)
    v = np.zeros_like(x)

    for i in range(startiter, startiter + maxiter):
        g = jac(x, *args)

        if callback and callback(x):
            break

        m = (1 - beta1) * g + beta1 * m  # first  moment estimate.
        v = (1 - beta2) * (g**2) + beta2 * v  # second moment estimate.
        mhat = m / (1 - beta1**(i + 1))  # bias correction.
        vhat = v / (1 - beta2**(i + 1))
        x = x - learning_rate * mhat / (np.sqrt(vhat) + eps)

    i += 1
    return OptimizeResult(x=x, fun=fun(x, *args), jac=g, nit=i, nfev=i, success=True)

#Write a function that implements adam with a decaying adaptive learning rate
def adam_with_lr_decay(
    fun,
    x0,
    jac,
    args=(),
    init_learning_rate=0.2, #0.001,
    lr_decay_rate=0.6,
    lr_decay_steps=10000,
    lr_decay_type="exponential",
    beta1=0.9,
    beta2=0.999,
    eps=1e-8,
    startiter=0,
    maxiter=10000,
    callback=None,
    **kwargs):
    """scipy.optimize.minimize compatible implementation of ADAM with learning rate decay - [http://arxiv.org/pdf/1412.6980.pdf].
    Adapted from autograd/misc/optimizers.py.
    """
    x = x0
    m = np.zeros_like(x)
    v = np.zeros_like(x)
    lr = init_learning_rate

    for i in range(startiter, startiter + maxiter):
        g = jac(x, *args)

        if callback and callback(x):
            break

        m = (1 - beta1) * g + beta1 * m
        v = (1 - beta2) * (g**2) + beta2 * v
        mhat = m / (1 - beta1**(i + 1))
        vhat = v / (1 - beta2**(i + 1))
        if lr_decay_type == "exponential":
            lr = init_learning_rate * (lr_decay_rate ** (i/lr_decay_steps))
        elif lr_decay_type == "linear":
            lr = init_learning_rate/(i+1)
        elif lr_decay_type == "factor":
            lr *= lr_decay_rate
        
        x = x - lr * mhat / (np.sqrt(vhat) + eps)
        
    i += 1
    return OptimizeResult(x=x, fun=fun(x, *args), jac=g, nit=i, nfev=i, success=True)


def adamw_with_lr_decay(
    fun,
    x0,
    jac,
    args=(),
    init_learning_rate=0.04,
    lr_decay_rate=0.5,
    lr_decay_steps=200,
    lr_decay_type="exponential",
    weight_decay=0.01, # weight decay parameter for AdamW
    beta1=0.9,
    beta2=0.999,
    eps=1e-8,
    startiter=0,
    maxiter=10000,
    callback=None,
    **kwargs):
    """scipy.optimize.minimize compatible implementation of ADAMW without learning rate decay - [http://arxiv.org/pdf/1412.6980.pdf].
    Adapted from autograd/misc/optimizers.py.
    """
    
    x = x0
    m = np.zeros_like(x)
    v = np.zeros_like(x)
    lr = init_learning_rate

    for i in range(startiter, startiter + maxiter):
        g = jac(x, *args)

        if callback and callback(x):
            break

        m = (1 - beta1) * g + beta1 * m  # momentum
        v = (1 - beta2) * (g**2) + beta2 * v  # RMSProp

        mhat = m / (1 - beta1**(i + 1))  # bias correction
        vhat = v / (1 - beta2**(i + 1))  # bias correction

        if lr_decay_type == "exponential":
            lr = init_learning_rate * (lr_decay_rate ** (i/lr_decay_steps))
        elif lr_decay_type == "linear":
            lr = init_learning_rate/(i+1)
        elif lr_decay_type == "factor":
            lr *= lr_decay_rate

        step_size = lr * mhat / (np.sqrt(vhat) + eps)

        x = (1 - weight_decay) * x - step_size  # AdamW update rule
        
    i += 1
    return OptimizeResult(x=x, fun=fun(x, *args), jac=g, nit=i, nfev=i, success=True)

                    

In [60]:
def initialize_circuit_params(qubits, params_init_mode):
    """
    Initialize the parameters of the circuit.
    
    Args:
    ----------------
        qubits [list]: list of qubits
        params_init_mode [str]: mode of initialization of the parameters.
            "zero": all parameters are initialized to zero
            "normal_random": all parameters are initialized to a random value drawn from a normal distribution
    
    Returns:
    ----------------
        init_circuit_params [ndarray]: array of the initial parameters of the circuit
    """
    
    init_circuit_params = None
    
    if params_init_mode == "zero":
        init_phi_params = np.zeros(8*len(qubits))
        init_theta_params = np.zeros(4*len(qubits))
        init_circuit_params = np.concatenate([init_phi_params, init_theta_params])
    elif params_init_mode == "normal_random":
        #init_phi_params = np.random.normal(-np.pi/2, np.pi/2, 8*len(qubits))	
        #init_theta_params = np.random.normal(-np.pi/2, np.pi/2, 4*len(qubits))
        init_phi_params = np.random.normal(-0.02, 0.02, 8*len(qubits))	
        init_theta_params = np.random.normal(-0.02, 0.02, 4*len(qubits))
        init_circuit_params = np.concatenate([init_phi_params, init_theta_params])
    return init_circuit_params


In [61]:
from scipy.optimize import minimize
import time


def meta_vqe_energy_loss(circuit_params, qubits, hamilt_params_training_set, n_meas_reps=1000):
    """
    Compute the energy loss of the meta_vqe circuit.
    
    Args:
    ----------------
        circuit_params [ndarray]: array of the parameters of the circuit
        qubits [list]: list of qubits
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians
        n_meas_reps [int]: number of measurement repetitions

    Returns:
    ----------------
        energy_loss [float]: energy loss of the meta_vqe circuit
    """
    
    energy_loss = 0
    
    #start_time = time.time()
    for hamilt_params_sample in hamilt_params_training_set:
        #energy_loss += meas_hamilt_expectation(qubits, circuit_params, hamilt_params_sample, n_meas_reps)
        energy_loss += direct_hamilt_expectation(qubits, circuit_params, hamilt_params_sample)
    
    #print("Time taken for the meta_vqe loss evaluation: ", time.time() - start_time)
    
    return energy_loss


def train_meta_vqe(qubits, hamilt_params_training_set, init_circuit_params=None, params_init_mode="normal_random", n_meas_reps=1000):
    """
    Train the meta_vqe circuit and return the trained PQC parameters.
    
    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians
        init_circuit_params [ndarray]: array of the initial parameters of the circuit
        params_init_mode [str]: mode of initialization of the parameters.

    Returns:
    ----------------
        trained_circuit_params [ndarray]: array of the trained parameters of the circuit
    """
    
    if init_circuit_params is None:
        init_circuit_params = initialize_circuit_params(qubits, params_init_mode)
    
    #train_loss_func = lambda circuit_params: meta_vqe_energy_loss(circuit_params, hamilt_params_training_set = hamilt_params_training_set)

    def callbackF(Xi):
        global Nfeval
        print("Iter: ", Nfeval, " Loss on train set: ", meta_vqe_energy_loss(Xi, qubits, hamilt_params_training_set, n_meas_reps=1000))
        Nfeval += 1

    print("Train loss should converge to ", sum(exact_hamilt_GS_energy(qubits, hamilt_params_training_set[i]) for i in range(len(hamilt_params_training_set))))
    print("Starting training...")
    
    loss_args = (qubits, hamilt_params_training_set, n_meas_reps)
    meta_vqe_loss_grad_fun = prepare_loss_gradient_approx(meta_vqe_energy_loss, stochastic_approx=True, stoch_shift_size_c=0.01)
    #meta_vqe_loss_grad_fun = prepare_loss_gradient_approx(lambda x: meta_vqe_energy_loss(x, *loss_args), stochastic_approx=True, stoch_shift_size_c=0.001)
    
    optim_result = minimize(meta_vqe_energy_loss, #train_loss_func, 
                            args=loss_args,
                            x0=init_circuit_params, 
                            method='L-BFGS-B', #adamw_with_lr_decay, #'L-BFGS-B', #'COBYLA', #adam_with_lr_decay,,#'COBYLA', 
                            callback=callbackF,
                            #jac=meta_vqe_loss_grad_fun, #None,
                            options={'maxiter': 12000},
                            )
    print(optim_result)
    #print(optim_result.fun)
    return optim_result.x

def evaluate_meta_vqe(qubits, circuit_params, hamilt_params_test_set, n_meas_reps=1000):
    """
    Evaluate the meta_vqe circuit on a test set of hamiltonian parameters.

    Args:
    ----------------
        qubits [list]: list of qubits
        circuit_params [ndarray]: array of the parameters of the circuit
        hamilt_params_test_set [list]: list of the parameters of the hamiltonians
        n_meas_reps [int]: number of measurement repetitions
    
    Returns:
    ----------------
        energy_expectations [list]: list of the energy expectations
        exact_energy_expectations [list]: list of the exact energy expectations
        abs_energy_errors [list]: list of the absolute energy errors
    """
    
    #Evaluate meta-VQE on test set:
    energy_expectations = []
    exact_energy_expectations = []
    abs_energy_errors = []
    
    #Infere on test set:
    for hamilt_params_sample in hamilt_params_test_set:
        #sample_energy_expect = meas_hamilt_expectation(qubits, circuit_params, hamilt_params_sample, n_meas_reps)
        sample_energy_expect =  direct_hamilt_expectation(qubits, circuit_params, hamilt_params_sample)
        energy_expectations.append(sample_energy_expect)
        sample_exact_energy_expect = exact_hamilt_GS_energy(qubits, hamilt_params_sample)
        exact_energy_expectations.append(sample_exact_energy_expect)
        abs_energy_errors.append(np.abs(sample_energy_expect - sample_exact_energy_expect))
    
    print("Meta-VQE absolute energy errors: ", abs_energy_errors)
    return np.array(energy_expectations), np.array(abs_energy_errors)

def meta_vqe(qubits, hamilt_params_training_set, hamilt_params_test_set):
    """
    Convenience wrapper to train and evaluate the meta_vqe circuit.

    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians
        hamilt_params_test_set [list]: list of the parameters of the hamiltonians
    
    Returns:
    ----------------
        energy_expectations [list]: list of the energy expectations
        abs_energy_errors [list]: list of the absolute energy errors
    """
    
    #Train meta-VQE:
    opt_circuit_params = train_meta_vqe(qubits, hamilt_params_training_set, params_init_mode="normal_random", n_meas_reps=1000)
    
    #Evaluate meta-VQE on test set:
    energy_expectations, abs_energy_errors = evaluate_meta_vqe(qubits, opt_circuit_params, hamilt_params_test_set, n_meas_reps=1000)
    
    return energy_expectations, abs_energy_errors

In [62]:
qubits = cirq.LineQubit.range(8)

Nfeval = 1  

train_set_size = 10
min_hamilt_params = {"lambda": 0.75, "Delta": -1.1}
max_hamilt_params = {"lambda": 0.75, "Delta": 1.1}

train_set = construct_training_set(train_set_size=train_set_size, mode="equidistant")
test_set = construct_test_set(test_set_size=100, mode="uniform_random", min_hamilt_params=min_hamilt_params, max_hamilt_params=max_hamilt_params)

meta_vqe(qubits, train_set, test_set)

Train loss should converge to  -125.38941301973591
Starting training...
Iter:  1  Loss on train set:  59.494582553396995
Iter:  2  Loss on train set:  59.49457959643691
Iter:  3  Loss on train set:  59.494579587651344
      fun: 59.494579587651344
 hess_inv: <96x96 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.58587849e+01, -3.51306149e+01, -1.96685995e+01, -2.95134065e+01,
        1.47565515e-02, -3.03259640e-03,  8.22808488e-03, -4.21280788e-03,
        1.48186776e+02, -3.32231309e-01,  1.48234312e+02,  1.48186560e+02,
        1.50261770e+02,  1.50262547e+02,  1.75834742e+00,  1.48169026e+02,
        6.07300876e-03, -1.23804895e+00, -1.57233870e-01, -1.42883749e-01,
       -1.93480787e-02, -6.00171290e-01, -1.66971859e-01, -1.24271082e-01,
       -1.25258026e-01, -9.13205156e-01, -4.53667326e-01, -4.85259477e-01,
       -2.03699102e-01, -4.02663147e-01, -2.86456512e+00, -2.86354691e+00,
       -4.47456280e+01, -4.47206581e+01, -4.47439092e+01, -4.42845383e+01,
       -

(array([ 5.95975633e+00,  8.48638438e+00,  1.09222120e+01, -2.76768919e+00,
         2.71404077e+00,  8.40460942e+00, -2.19575832e+00,  8.05126325e+00,
         3.83541801e+00,  1.21306542e+01,  1.19844853e+01,  1.10973213e+01,
         1.33270742e+01,  1.19817039e+01,  8.93225996e+00,  1.44826683e+01,
        -4.45125480e-01,  1.21602716e+01,  6.82379002e+00,  1.18058331e+01,
        -3.07230453e-01,  1.07385088e+01,  2.32441097e-01,  1.18421607e+01,
         1.43093290e+01,  5.56413760e+00,  4.73026747e+00,  4.54056022e-01,
         6.80044966e+00,  1.40038845e+01, -6.69848808e-01,  1.40412372e+01,
         3.74036362e-01,  7.64544743e+00,  9.42941935e+00, -6.85270910e-01,
        -1.88554007e-01,  2.20831752e-01,  1.79093385e+00,  6.63896040e-01,
         1.79448640e+00,  4.13082200e+00,  2.79027279e+00,  9.93236153e+00,
         8.38177644e+00,  6.31200027e+00, -6.85952960e-01,  1.25004604e+01,
         2.84715528e+00,  6.81647347e+00,  1.33365009e+01,  6.91732633e+00,
        -1.8

In [255]:
def staged_meta_vqe_energy_loss(circuit_params, qubits, frozen_circuit_params,insertion_pointer, hamilt_params_training_set, n_meas_reps=1000):
    """
    Compute the energy loss of the meta_vqe circuit for a staged meta-vqe, i.e. a meta-vqe trained layer-by-layer.
    
    Args:
    ----------------
        circuit_params [ndarray]: array of the parameters of the circuit
        qubits [list]: list of qubits
        frozen_circuit_params [ndarray]: array of the parameters of the frozen part of the circuit
        insertion_pointer [int]: index of the insertion point of the circuit_params
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians
        n_meas_reps [int]: number of measurement repetitions
    
    Returns:
    ----------------
        energy_loss [float]: energy loss of the meta_vqe circuit
    """
    
    energy_loss = 0
    
    #start_time = time.time()
    for hamilt_params_sample in hamilt_params_training_set:
        full_circuit_params = np.insert(frozen_circuit_params, insertion_pointer, circuit_params)
        #energy_loss += meas_hamilt_expectation(qubits, circuit_params, hamilt_params_sample, n_meas_reps)
        energy_loss += direct_hamilt_expectation(qubits, full_circuit_params, hamilt_params_sample)
    
    #print("Time taken for the meta_vqe loss evaluation: ", time.time() - start_time)
    
    return energy_loss

def train_staged_meta_vqe(qubits, hamilt_params_training_set, init_circuit_params=None, params_init_mode="normal_random", n_meas_reps=1000):
    """
    Train the meta_vqe circuit for a staged meta-vqe, i.e. a meta-vqe trained layer-by-layer.
    
    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians
        init_circuit_params [ndarray]: array of the parameters of the circuit
        params_init_mode [str]: mode of initialization of the circuit parameters
        n_meas_reps [int]: number of measurement repetitions
        
    Returns:
    ----------------
        opt_circuit_params [ndarray]: array of the parameters of the optimized circuit
    """
    
    if init_circuit_params is None:
        init_circuit_params = initialize_circuit_params(qubits, params_init_mode)


    num_var_encoding_layers, num_var_processing_layers = 2, 2 #Default for the paper PQC
    params_per_processing_layer = 2 * len(qubits)
    params_per_encoding_layer = 2 * params_per_processing_layer
    layer_sizes =  [params_per_encoding_layer for i in range(num_var_encoding_layers)] + [params_per_processing_layer for i in range(num_var_processing_layers)]
    print("Layer sizes: ", layer_sizes)

    print("Train loss should converge to ", sum(exact_hamilt_GS_energy(qubits, hamilt_params_training_set[i]) for i in range(len(hamilt_params_training_set))))
    print("Starting training...")


    meta_vqe_loss_grad_fun = prepare_loss_gradient_approx(staged_meta_vqe_energy_loss, stochastic_approx=True, stoch_shift_size_c=0.01)


    optim_circuit_params = init_circuit_params
    for repetit in range(4):
        print("Running repetition: ", repetit)
        for lay_i in reversed(range(len(layer_sizes))):
            
            unfreeze_steps = 200
            
            insertion_pointer = sum(layer_sizes[:lay_i]) #Index of the first trainable parameter in the complete circuit parameter vector
            frozen_circuit_params = np.concatenate([optim_circuit_params[:insertion_pointer], optim_circuit_params[insertion_pointer + layer_sizes[lay_i]:]])
            trainable_circuit_params = optim_circuit_params[insertion_pointer:insertion_pointer + layer_sizes[lay_i]]
            print("Unfrozen params: ", len(trainable_circuit_params), " Frozen params: ", len(frozen_circuit_params))
            
            def callbackF(Xi):
                global Nfeval
                print("Iter: ", Nfeval, " Loss on train set: ", staged_meta_vqe_energy_loss(Xi, qubits, frozen_circuit_params, insertion_pointer, hamilt_params_training_set, n_meas_reps=1000))
                Nfeval += 1
            
            loss_args = (qubits, frozen_circuit_params, insertion_pointer, hamilt_params_training_set, n_meas_reps)
            
            
            optim_result = minimize(staged_meta_vqe_energy_loss,
                                    args=loss_args, 
                                    x0=trainable_circuit_params, 
                                    method=adam, #'L-BFGS-B', #'COBYLA', #adam_with_lr_decay,,#'COBYLA', 
                                    callback=callbackF,
                                    jac=meta_vqe_loss_grad_fun, #None,
                                    options={'maxiter': unfreeze_steps},
                                    )
            print(optim_result)
            optim_circuit_params = np.insert(frozen_circuit_params,insertion_pointer, optim_result.x)        
        
    #print(optim_result.fun)
    return optim_circuit_params

def staged_meta_vqe(qubits, hamilt_params_training_set, hamilt_params_test_set):
    """
    Convenience function for training and evaluating a staged meta-vqe, i.e. a meta-vqe trained layer-by-layer.
    
    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians
        hamilt_params_test_set [list]: list of the parameters of the hamiltonians
        
    Returns:
    ----------------
        energy_expectations [list]: list of the energy expectations
        abs_energy_errors [list]: list of the absolute energy errors
    """
    
    #Train meta-VQE:
    opt_circuit_params = train_staged_meta_vqe(qubits, hamilt_params_training_set, params_init_mode="normal_random", n_meas_reps=1000)
    
    #Evaluate meta-VQE on test set:
    #energy_expectations, abs_energy_errors = evaluate_meta_vqe(qubits, opt_circuit_params, hamilt_params_test_set, n_meas_reps=1000)
    
    ##return energy_expectations, abs_energy_errors


qubits = cirq.LineQubit.range(8)

Nfeval = 1  

train_set_size = 10
min_hamilt_params = {"lambda": 0.75, "Delta": -1.1}
max_hamilt_params = {"lambda": 0.75, "Delta": 1.1}

train_set = construct_training_set(train_set_size=train_set_size, mode="equidistant")
test_set = construct_test_set(test_set_size=100, mode="uniform_random", min_hamilt_params=min_hamilt_params, max_hamilt_params=max_hamilt_params)

staged_meta_vqe(qubits, train_set, test_set)

Layer sizes:  [32, 32, 16, 16]
Train loss should converge to  -125.38941301973591
Starting training...
Running repetition:  0
Unfrozen params:  16  Frozen params:  80
Iter:  1  Loss on train set:  59.59967672618967
Iter:  2  Loss on train set:  59.394431836798006
Iter:  3  Loss on train set:  59.11632385704006
Iter:  4  Loss on train set:  58.83675465883579
Iter:  5  Loss on train set:  58.40185787056015
Iter:  6  Loss on train set:  57.81619904253222
Iter:  7  Loss on train set:  57.14359060399025
Iter:  8  Loss on train set:  56.255244123437244
Iter:  9  Loss on train set:  55.30922542503967
Iter:  10  Loss on train set:  54.05975351500556
Iter:  11  Loss on train set:  52.777767512180716
Iter:  12  Loss on train set:  51.44109069706958
Iter:  13  Loss on train set:  50.11264276976726
Iter:  14  Loss on train set:  48.835835568424656
Iter:  15  Loss on train set:  47.60791189628557
Iter:  16  Loss on train set:  46.3996651584349
Iter:  17  Loss on train set:  45.22996555627721
Iter: 

In [210]:
def dropout_meta_vqe_energy_loss(circuit_params, qubits, frozen_circuit_params,insertion_pointers, inverted_insertion_pointers, hamilt_params_training_set, n_meas_reps=1000):
    """
    Computes the energy loss of a meta-vqe with parameter dropout, i.e. a meta-vqe where the parameters of the circuit are randomly dropped out in each training step.
    
    Args:
    ----------------
        circuit_params [np.array]: array of the circuit parameters
        qubits [list]: list of qubits
        frozen_circuit_params [np.array]: array of the frozen circuit parameters
        insertion_pointers [list]: list of the insertion pointers
        inverted_insertion_pointers [list]: list of the inverted insertion pointers
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians
        n_meas_reps [int]: number of measurement repetitions
        
    Returns:
    ----------------
        energy_loss [float]: energy loss of the meta-vqe
    """
    
    energy_loss = 0
    full_circuit_params = np.zeros(len(circuit_params) + len(frozen_circuit_params))
    full_circuit_params[insertion_pointers] = circuit_params
    full_circuit_params[inverted_insertion_pointers] = frozen_circuit_params
    
    #start_time = time.time()
    for hamilt_params_sample in hamilt_params_training_set:
        #energy_loss += meas_hamilt_expectation(qubits, circuit_params, hamilt_params_sample, n_meas_reps)
        energy_loss += direct_hamilt_expectation(qubits, full_circuit_params, hamilt_params_sample)
    
    #print("Time taken for the meta_vqe loss evaluation: ", time.time() - start_time)
    
    return energy_loss

def train_dropout_meta_vqe(qubits, hamilt_params_training_set, init_circuit_params=None, params_init_mode="normal_random", n_meas_reps=1000):
    """
    Trains a meta-vqe with parameter dropout, i.e. a meta-vqe where the parameters of the circuit are randomly dropped out in each training step.
    
    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians
        init_circuit_params [np.array]: array of the initial circuit parameters
        params_init_mode [str]: mode for initializing the circuit parameters
        n_meas_reps [int]: number of measurement repetitions
    
    Returns:
    ----------------
        optim_circuit_params [np.array]: array of the optimized circuit parameters
    """
    
    if init_circuit_params is None:
        init_circuit_params = initialize_circuit_params(qubits, params_init_mode)


    print("Train loss should converge to ", sum(exact_hamilt_GS_energy(qubits, hamilt_params_training_set[i]) for i in range(len(hamilt_params_training_set))))
    print("Starting training...")


    meta_vqe_loss_grad_fun = prepare_loss_gradient_approx(dropout_meta_vqe_energy_loss, stochastic_approx=True, stoch_shift_size_c=0.01)

    optim_circuit_params = init_circuit_params
    dropout_rate = 0.8
    
    for repetit in range(80):
        shuffled_inds = np.random.permutation(len(optim_circuit_params)) #np.random.randint(0, len(optim_circuit_params), size=int((1-dropout_rate)*len(optim_circuit_params)))
        insertion_pointers, inverted_insertion_pointers = shuffled_inds[:int((1-dropout_rate)*len(optim_circuit_params))], shuffled_inds[int((1-dropout_rate)*len(optim_circuit_params)):]
        trainable_circuit_params = optim_circuit_params[insertion_pointers]
        frozen_circuit_params = optim_circuit_params[inverted_insertion_pointers]
        
        print("Unfrozen params: ", len(trainable_circuit_params), " Frozen params: ", len(frozen_circuit_params))
        
        def callbackF(Xi):
            global Nfeval
            print("Iter: ", Nfeval, " Loss on train set: ", dropout_meta_vqe_energy_loss(Xi, qubits, frozen_circuit_params, insertion_pointers, inverted_insertion_pointers, hamilt_params_training_set, n_meas_reps=1000))
            Nfeval += 1
        
        loss_args = (qubits, frozen_circuit_params, insertion_pointers, inverted_insertion_pointers, hamilt_params_training_set, n_meas_reps)
        
        #meta_vqe_loss_grad_fun = prepare_loss_gradient_approx(lambda x: dropout_meta_vqe_energy_loss(x, *loss_args), stochastic_approx=True, stoch_shift_size_c=0.001)
    
        optim_result = minimize(dropout_meta_vqe_energy_loss,
                                args=loss_args, 
                                x0=trainable_circuit_params, 
                                method="COBYLA", #adam, #'L-BFGS-B', #'COBYLA', #adam_with_lr_decay,,#'COBYLA', 
                                callback=callbackF,
                                jac=meta_vqe_loss_grad_fun, #None,
                                options={'maxiter': 50},
                                )
        print(optim_result)
        optim_circuit_params[insertion_pointers] = optim_result.x     
        
    #print(optim_result.fun)
    return optim_circuit_params

def dropout_meta_vqe(qubits, hamilt_params_training_set, hamilt_params_test_set):
    """
    Convenience function for training and evaluating a dropout meta-vqe, i.e. a meta-vqe where the parameters of the circuit are randomly dropped out in each training step.
    
    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians in the training set
        hamilt_params_test_set [list]: list of the parameters of the hamiltonians in the test set
    
    Returns:
    ----------------
        energy_expectations [list]: list of the energy expectations of the hamiltonians in the test set
        abs_energy_errors [list]: list of the absolute energy errors of the hamiltonians in the test set
    """
    
    #Train meta-VQE:
    opt_circuit_params = train_dropout_meta_vqe(qubits, hamilt_params_training_set, params_init_mode="normal_random", n_meas_reps=1000)
    
    #Evaluate meta-VQE on test set:
    #energy_expectations, abs_energy_errors = evaluate_meta_vqe(qubits, opt_circuit_params, hamilt_params_test_set, n_meas_reps=1000)
    
    ##return energy_expectations, abs_energy_errors


qubits = cirq.LineQubit.range(8)

Nfeval = 1  

train_set_size = 10
min_hamilt_params = {"lambda": 0.75, "Delta": -1.1}
max_hamilt_params = {"lambda": 0.75, "Delta": 1.1}

train_set = construct_training_set(train_set_size=train_set_size, mode="equidistant")
test_set = construct_test_set(test_set_size=100, mode="uniform_random", min_hamilt_params=min_hamilt_params, max_hamilt_params=max_hamilt_params)

dropout_meta_vqe(qubits, train_set, test_set)

Train loss should converge to  -65.56662081055886
Starting training...
Unfrozen params:  9  Frozen params:  39


c:\Users\trothe\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:550: RuntimeWarning: Method COBYLA does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,


Iter:  1  Loss on train set:  29.848294994033672
Iter:  2  Loss on train set:  20.67942723111402
Iter:  3  Loss on train set:  19.03461001974393
Iter:  4  Loss on train set:  19.034599333402895
Iter:  5  Loss on train set:  21.241273359923063
Iter:  6  Loss on train set:  19.034597154249738
Iter:  7  Loss on train set:  19.03459618306855
Iter:  8  Loss on train set:  19.03643008354366
Iter:  9  Loss on train set:  19.034594614883904
Iter:  10  Loss on train set:  8.356113272396618
Iter:  11  Loss on train set:  1.666650185429389
Iter:  12  Loss on train set:  -1.0119488578395919
Iter:  13  Loss on train set:  -2.3579173912451346
Iter:  14  Loss on train set:  -4.329720804531213
Iter:  15  Loss on train set:  10.724485917052926
Iter:  16  Loss on train set:  -4.325915153598629
Iter:  17  Loss on train set:  -16.115202673701855
Iter:  18  Loss on train set:  -28.4491817847101
Iter:  19  Loss on train set:  -33.21124851403562
Iter:  20  Loss on train set:  -28.83240464624062
Iter:  21  Lo

c:\Users\trothe\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:550: RuntimeWarning: Method COBYLA does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,


Iter:  52  Loss on train set:  -34.48793064031567
Iter:  53  Loss on train set:  -39.15146736256149
Iter:  54  Loss on train set:  -39.22290789064992
Iter:  55  Loss on train set:  -37.143018372144155
Iter:  56  Loss on train set:  -35.27015426760535
Iter:  57  Loss on train set:  -24.745621442239607
Iter:  58  Loss on train set:  -24.663650507559947
Iter:  59  Loss on train set:  -35.273326335987626
Iter:  60  Loss on train set:  -37.08684393479913
Iter:  61  Loss on train set:  -27.39172405601239
Iter:  62  Loss on train set:  -37.805223467728084
Iter:  63  Loss on train set:  -38.48325974365437
Iter:  64  Loss on train set:  -38.37852596392495
Iter:  65  Loss on train set:  -39.26807529423103
Iter:  66  Loss on train set:  -38.48882465533058
Iter:  67  Loss on train set:  -39.31020925024282
Iter:  68  Loss on train set:  -38.73600861179296
Iter:  69  Loss on train set:  -39.24038404047503
Iter:  70  Loss on train set:  -36.80836947991742
Iter:  71  Loss on train set:  -39.0623527354

...

Iter:  3928  Loss on train set:  -52.439913947358534
Iter:  3929  Loss on train set:  -49.683337601446
Iter:  3930  Loss on train set:  -41.31988420806464
Iter:  3931  Loss on train set:  -50.6296111198937
Iter:  3932  Loss on train set:  -47.66018041838137
Iter:  3933  Loss on train set:  -52.439915401497906
Iter:  3934  Loss on train set:  -52.43991676202156
Iter:  3935  Loss on train set:  -40.04006709878055
Iter:  3936  Loss on train set:  -52.43991588551357
Iter:  3937  Loss on train set:  -40.388891033911484
Iter:  3938  Loss on train set:  -49.20382087702217
Iter:  3939  Loss on train set:  -51.83475133780131
Iter:  3940  Loss on train set:  -52.439915893616565
Iter:  3941  Loss on train set:  -52.24472373576927
Iter:  3942  Loss on train set:  -52.405470308531655
Iter:  3943  Loss on train set:  -52.074913552694824
Iter:  3944  Loss on train set:  -52.43991279358243
Iter:  3945  Loss on train set:  -51.55190860309216
Iter:  3946  Loss on train set:  -52.439918514710456
Iter:  3

c:\Users\trothe\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:550: RuntimeWarning: Method COBYLA does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,


Iter:  3979  Loss on train set:  -52.440580785304206
Iter:  3980  Loss on train set:  -42.560649017401055
Iter:  3981  Loss on train set:  -47.67027013384785
Iter:  3982  Loss on train set:  -39.66088203020692
Iter:  3983  Loss on train set:  -43.55753740013025
Iter:  3984  Loss on train set:  -48.43232535938608
Iter:  3985  Loss on train set:  -40.166991932884635
Iter:  3986  Loss on train set:  -40.578096656537014
Iter:  3987  Loss on train set:  -41.271009467310435
Iter:  3988  Loss on train set:  -40.62577678806234
Iter:  3989  Loss on train set:  -49.00068514023417
Iter:  3990  Loss on train set:  -51.75086638737793
Iter:  3991  Loss on train set:  -52.440572920032295
Iter:  3992  Loss on train set:  -52.08519292369413
Iter:  3993  Loss on train set:  -52.235589874899965
Iter:  3994  Loss on train set:  -51.60602360531109
Iter:  3995  Loss on train set:  -52.36378058772272
Iter:  3996  Loss on train set:  -51.688995902163235
Iter:  3997  Loss on train set:  -52.33503536479266
Iter

c:\Users\trothe\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:550: RuntimeWarning: Method COBYLA does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,


Iter:  4030  Loss on train set:  -44.319672902971924
Iter:  4031  Loss on train set:  -46.22638852273992
Iter:  4032  Loss on train set:  -46.26598037350667
Iter:  4033  Loss on train set:  -41.043115878636044
Iter:  4034  Loss on train set:  -48.21817494861368
Iter:  4035  Loss on train set:  -51.926474918605315
Iter:  4036  Loss on train set:  -39.90411612464985
Iter:  4037  Loss on train set:  -52.44419093646042
Iter:  4038  Loss on train set:  -46.804216344967024
Iter:  4039  Loss on train set:  -49.783989149482046
Iter:  4040  Loss on train set:  -50.34402079375843
Iter:  4041  Loss on train set:  -52.03335882340769
Iter:  4042  Loss on train set:  -51.03822817885213
Iter:  4043  Loss on train set:  -52.1493940261313
Iter:  4044  Loss on train set:  -47.729837436007095
Iter:  4045  Loss on train set:  -52.40249859744721
Iter:  4046  Loss on train set:  -47.51092936514785
Iter:  4047  Loss on train set:  -52.3058764948053
Iter:  4048  Loss on train set:  -52.44418488221676
Iter:  4

### The "classical" VQE

In [ ]:
def vqe_energy_loss(circuit_params,qubits, c_hamilt_params, n_meas_reps=1000):
    """
    Compute the loss function of a VQE
    
    Args:
    ----------------
        circuit_params [np.array]: parameters of the circuit
        qubits [list]: list of qubits
        c_hamilt_params [dict]: parameters of the hamiltonian
        n_meas_reps [int]: number of measurement repetitions
    
    Returns:
    ----------------
        loss [float]: loss function value
    """
    
    return direct_hamilt_expectation(qubits, circuit_params, c_hamilt_params)
    #return meas_hamilt_expectation(qubits, circuit_params, c_hamilt_params, n_meas_reps)

def optimize_vqe(qubits,hamilt_params, init_circuit_params, n_meas_reps=1000):
    """
    Optimize a VQE
    
    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params [dict]: parameters of the hamiltonian
        init_circuit_params [np.array]: initial parameters of the circuit
        n_meas_reps [int]: number of measurement repetitions
    
    Returns:
    ----------------
        opt_result [OptimizeResult]: result of the optimization
    """
    
    def callbackF(Xi):
        global Nfeval
        print("Iter: ", Nfeval, " Loss on train set: ", vqe_energy_loss(Xi, qubits, hamilt_params, n_meas_reps))
        Nfeval += 1
    
    print("Energy should converge to ", exact_hamilt_GS_energy(qubits, hamilt_params))
    print("Starting VQE minimization...")
    
    opt_result = minimize(vqe_energy_loss,
                            args=(qubits, hamilt_params, n_meas_reps),
                            x0=init_circuit_params,
                            method='COBYLA',
                            callback=callbackF,
                            )
    
    print(opt_result)
    return opt_result.x

def evaluate_vqe(qubits, hamilt_params_set, init_circuit_params=None, params_init_mode="normal_random", n_meas_reps=1000):
    """
    Evaluate a VQE on a set of hamiltonians
    
    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params_set [list]: list of the parameters of the hamiltonians
        init_circuit_params [np.array]: initial parameters of the circuit
        params_init_mode [str]: mode for initializing the circuit parameters
        n_meas_reps [int]: number of measurement repetitions
    
    Returns:
    ----------------
        energy_expectations [np.array]: energy expectations of the hamiltonians
        exact_energy_expectations [np.array]: exact energy expectations of the hamiltonians
        abs_energy_errors [np.array]: absolute energy errors of the hamiltonians
    """
    
    energy_expectations = []
    exact_energy_expectations = []
    abs_energy_errors = []
    
    for hamilt_params_sample in hamilt_params_set:
        if init_circuit_params is None:
            init_circuit_params = initialize_circuit_params(qubits, params_init_mode)
    
        opt_circuit_params = optimize_vqe(qubits, hamilt_params_sample, params_init_mode, n_meas_reps)
        
        energy_expectation = direct_hamilt_expectation(qubits, opt_circuit_params, hamilt_params_sample, n_meas_reps)
        exact_energy_expectation = exact_hamilt_GS_energy(qubits, hamilt_params_sample)
        abs_energy_error = np.abs(energy_expectation - exact_energy_expectation)
        energy_expectations.append(energy_expectation)
        exact_energy_expectations.append(exact_energy_expectation)
        abs_energy_errors.append(abs_energy_error)
    
    return np.array(energy_expectations), np.array(abs_energy_errors)
    
def vqe(qubits, hamilt_params_set):
    """
    Convenience function for wrapping VQE routines
    
    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params_set [list]: list of the parameters of the hamiltonians
        
    Returns:
    ----------------
        energy_expectations [list]: list of the energy expectations of the hamiltonians in the test set
        abs_energy_errors [list]: list of the absolute energy errors of the hamiltonians in the test set
    """
    
    energy_expectations, abs_energy_errors = evaluate_vqe(qubits, hamilt_params_set, params_init_mode="normal_random", n_meas_reps=1000)
    return energy_expectations, abs_energy_errors

In [145]:
qubits = cirq.LineQubit.range(8)

Nfeval = 1  

vqe_hamilt_params = {"lambda": 0.75, "Delta": 0.1}

vqe(qubits,hamilt_params=vqe_hamilt_params, params_init_mode="normal_random", n_meas_reps=100000)

Energy should converge to  -11.403722989559629
Starting VQE minimization...
Iter:  1  Loss on train set:  3.3393581853085332
Iter:  2  Loss on train set:  3.363206600542764
Iter:  3  Loss on train set:  3.3393580730270656
Iter:  4  Loss on train set:  3.3129355127064017
Iter:  5  Loss on train set:  3.312935295917041
Iter:  6  Loss on train set:  3.3336963990519557
Iter:  7  Loss on train set:  3.3129350972611995
Iter:  8  Loss on train set:  3.355789620554316
Iter:  9  Loss on train set:  3.3129353011139546
Iter:  10  Loss on train set:  3.2672526606345387
Iter:  11  Loss on train set:  3.267253519759116
Iter:  12  Loss on train set:  3.31256952664296
Iter:  13  Loss on train set:  3.2672526724716793
Iter:  14  Loss on train set:  3.2702085574577957
Iter:  15  Loss on train set:  3.267252822905118
Iter:  16  Loss on train set:  3.2707130465546452
Iter:  17  Loss on train set:  3.3448961532893433
Iter:  18  Loss on train set:  3.3482541574712372
Iter:  19  Loss on train set:  3.2403399

### The Opt-Meta-VQE

In [ ]:
def opt_meta_vqe(qubits, hamilt_params_training_set, hamilt_params_test_set, n_meas_reps=1000):
    """
    Full routine to train and evaluate a opt-meta-VQE
    
    Args:
    ----------------
        qubits [list]: list of qubits
        hamilt_params_training_set [list]: list of the parameters of the hamiltonians in the training set
        hamilt_params_test_set [list]: list of the parameters of the hamiltonians in the test set
        n_meas_reps [int]: number of measurement repetitions
    
    Returns:
    ----------------
        energy_expectations [list]: list of the energy expectations of the hamiltonians in the test set
        abs_energy_errors [list]: list of the absolute energy errors of the hamiltonians in the test set
    """
    
    #Train meta-VQE:
    meta_vqe_opt_circuit_params = train_meta_vqe(qubits, hamilt_params_training_set, params_init_mode="normal_random")
    
    #Optimize a VQE on each test set sample:
    energy_expectations, abs_energy_errors = evaluate_vqe(qubits, hamilt_params_test_set, init_circuit_params=meta_vqe_opt_circuit_params)
    
    return energy_expectations, abs_energy_errors

In [ ]:
qubits = cirq.LineQubit.range(5)

Nfeval = 1

train_set_size = 10
min_hamilt_params = {"lambda": 0.75, "Delta": -1.1}
max_hamilt_params = {"lambda": 0.75, "Delta": 1.1}

train_set = construct_training_set(train_set_size=train_set_size, mode="equidistant")
test_set = construct_test_set(test_set_size=10, mode="uniform_random", min_hamilt_params=min_hamilt_params, max_hamilt_params=max_hamilt_params)

opt_meta_vqe(qubits, train_set, test_set, n_meas_reps=1000)


### Concluding experiments

In [ ]:
#Reproducing figure 2 of the meta-vqe paper: Absolute energy and ground state energy as a function of Delta
import matplotlib.pyplot as plt

fig, axes = plt.subfigure(2)
#axes[0].plot() #Exact GS energy
axes[0].plot() #VQE GS energy
axes[0].plot() #Meta-VQE GS energy
axes[0].plot() #Opt-Meta-VQE GS energy
axes[0].set_title("Ground State Energy")
axes[0].set_xlabel("Delta")
axes[0].set_ylabel("GS Energy")

axes[1].plot() #VQE Absolute Energy Error
axes[1].plot() #Meta-VQE Absolute Energy Error
axes[1].plot() #Opt-Meta-VQE Absolute Energy Error
axes[1].set_title("Absolute Energy Error")
axes[1].set_xlabel("Delta")
axes[1].set_ylabel("Absolute Energy Error")

fig.tight_layout()
fig.legend()

